In [ ]:
#@title Загружаем билиотеки
import os
import shutil
from shutil import copytree
import zipfile
import cv2
import albumentations as A
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import uuid
from PIL import Image
import re
import random

from google.colab import drive, files                                           # подключем диск, # Загрузка файлов                                                                  # импортируем модуль для удобной работы с файлами
import gdown
print(f"\x1b[34mБиблиотеки загружены.")

Библиотеки загружены.


In [ ]:
#@title Монтируем диск
try:
    drive.mount('/content/drive')
    print(f"\x1b[34mGoogle Drive успешно подключен.")
except Exception as e:
    print("Не удалось подключить Google Drive:", e)

Mounted at /content/drive
Google Drive успешно подключен.


In [ ]:
#@title Скачиваем файл данных
path_file = 'https://drive.google.com/uc?id=1h1YamoOHXuJhPT36OD2i4oiWvxjlnVyq' # Karate mixed 11 classes 562 Images _ crop X-X.zip
gdown.download(path_file, None, quiet = True)
!echo "Скачивание архива успешно завершено!"

Скачивание архива успешно завершено!


In [ ]:
#@title Копируем файл архива
# Укажите путь для архива
zip_file_path = '/content/drive/MyDrive/Colab Notebooks/Datas/Zips/Karate mixed 11 classes 562 Images _ crop X-X.zip'
send_zip = '/content/Karate mixed 11 classes 562 Images _ crop X-X.zip'
# Копирование
shutil.copy(zip_file_path, send_zip)

'/content/Karate mixed 11 classes 562 Images _ crop X-X.zip'

In [ ]:
#@title Разархивируем архив данных
!unzip -q "Karate mixed 11 classes 562 Images _ crop X-X.zip"
!echo "Распаковка архива успешно завершена!"

Распаковка архива успешно завершена!


In [ ]:
#@title Переименование файлов для обучения модели
def extract_prefix(folder_name):
    # Используем регулярное выражение для извлечения текста в скобках
    match = re.search(r'\((.*?)\)', folder_name)
    return match.group(1) if match else folder_name

def rename_files(folder_path):
    subfolders = [f.path for f in os.scandir(folder_path) if f.is_dir()]

    for subfolder in subfolders:
        # Получаем префикс из имени подпапки
        prefix = extract_prefix(os.path.basename(subfolder))

        # Получаем список файлов в текущей подпапке
        files = os.listdir(subfolder)

        # Перемешиваем список файлов
        random.shuffle(files)

        i = 1  # Начальное значение для номера файла

        for filename in files:
            old_filepath = os.path.join(subfolder, filename)

            unique_prefix = prefix.lower().replace(" ", "_")
            new_filename = f"{unique_prefix}_{i}.jpg"
            new_filepath = os.path.join(subfolder, new_filename)

            os.rename(old_filepath, new_filepath)
            print(f"Файл {filename} переименован в {new_filename}")

            i += 1  # Инкрементируем номер файла

# Переименование файлов в обучающей выборке
folder_path_train = "/content/Karate mixed 11 classes 562 Images _ crop X-X/Karate train"
rename_files(folder_path_train)

# Переименование файлов в тестовой выборке
folder_path_test = "/content/Karate mixed 11 classes 562 Images _ crop X-X/Karate test"
rename_files(folder_path_test)

Выходные данные были обрезаны до нескольких последних строк (5000).
Файл Shiko_dachi_ph_ft.jpg переименован в shiko_dachi_107.jpg
Файл Positions de base des pieds de karate 0975.jpg переименован в shiko_dachi_108.jpg
Файл KARATE HOJO UNDO 070301.jpg переименован в shiko_dachi_109.jpg
Файл Tsuguo Sakumoto 02955.jpg переименован в shiko_dachi_110.jpg
Файл maxresdefault (5).jpg переименован в shiko_dachi_111.jpg
Файл shiko-dachi-2-300x230_black.jpg переименован в shiko_dachi_112.jpg
Файл shiko-dachi-visione-prospettica.jpg переименован в shiko_dachi_113.jpg
Файл class 1 Kazoku Karate 17054.jpg переименован в shiko_dachi_114.jpg
Файл dynamic_stances_shiko.jpg переименован в shiko_dachi_115.jpg
Файл shiko-dess.jpg переименован в shiko_dachi_116.jpg
Файл KARATE AT HOME - 4   Karatebeginners2020 4012.jpg переименован в shiko_dachi_117.jpg
Файл 21185_html_63f35fef.jpg переименован в shiko_dachi_118.jpg
Файл Архивное видео клуба 02739.jpg переименован в shiko_dachi_119.jpg
Файл nHRdCw2NCQ.jpg п

In [ ]:
#@title Приведение картинок к размеру 224х224
def resize_and_save_images(folder_path, target_size=(224, 224)):
    subfolders = [f.path for f in os.scandir(folder_path) if f.is_dir()]

    # Создаем новую папку для измененных файлов
    new_folder_name = f"{os.path.basename(folder_path)}_224x224"
    new_folder_path = os.path.join(os.path.dirname(folder_path), new_folder_name)
    os.makedirs(new_folder_path, exist_ok=True)

    for subfolder in subfolders:
        prefix = extract_prefix(os.path.basename(subfolder))
        i = 1  # Начальное значение для номера файла

        # Создаем подпапку в новой папке
        new_subfolder_path = os.path.join(new_folder_path, os.path.basename(subfolder))
        os.makedirs(new_subfolder_path, exist_ok=True)

        for filename in os.listdir(subfolder):
            old_filepath = os.path.join(subfolder, filename)

            # Изменяем размер изображения
            try:
                img = Image.open(old_filepath)
                img = img.resize(target_size, Image.ANTIALIAS)

                unique_prefix = prefix.lower().replace(" ", "_")
                new_filename = f"{unique_prefix}_{i}.jpg"
                new_filepath = os.path.join(new_subfolder_path, new_filename)

                img.save(new_filepath)
                print(f"Файл {filename} изменен и сохранен в {new_filepath}")

                i += 1  # Инкрементируем номер файла
            except Exception as e:
                print(f"Ошибка при обработке файла {filename}: {str(e)}")

folder_path = "/content/Karate mixed 11 classes 562 Images _ crop X-X/Karate train"
resize_and_save_images(folder_path)

folder_path = "/content/Karate mixed 11 classes 562 Images _ crop X-X/Karate test"
resize_and_save_images(folder_path)

<ipython-input-6-92a096fdfa76>:24: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize(target_size, Image.ANTIALIAS)


Выходные данные были обрезаны до нескольких последних строк (5000).
Файл shiko_dachi_87.jpg изменен и сохранен в /content/Karate mixed 11 classes 562 Images _ crop X-X/Karate train_224x224/Шико Дачи (Shiko Dachi)/shiko_dachi_101.jpg
Файл shiko_dachi_21.jpg изменен и сохранен в /content/Karate mixed 11 classes 562 Images _ crop X-X/Karate train_224x224/Шико Дачи (Shiko Dachi)/shiko_dachi_102.jpg
Файл shiko_dachi_39.jpg изменен и сохранен в /content/Karate mixed 11 classes 562 Images _ crop X-X/Karate train_224x224/Шико Дачи (Shiko Dachi)/shiko_dachi_103.jpg
Файл shiko_dachi_116.jpg изменен и сохранен в /content/Karate mixed 11 classes 562 Images _ crop X-X/Karate train_224x224/Шико Дачи (Shiko Dachi)/shiko_dachi_104.jpg
Файл shiko_dachi_309.jpg изменен и сохранен в /content/Karate mixed 11 classes 562 Images _ crop X-X/Karate train_224x224/Шико Дачи (Shiko Dachi)/shiko_dachi_105.jpg
Файл shiko_dachi_393.jpg изменен и сохранен в /content/Karate mixed 11 classes 562 Images _ crop X-X/Kara

In [ ]:
#@title Архивация данных и копирование в целевую папку
# Архивируем папку с аугментированными изображениями
def create_zip_archive(zip_file_path, source_dirs):
    # Создание архива с аугментированными изображениями
    with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for source_dir in tqdm(source_dirs, desc = "Общий прогресс"):
            for dir_path, _, file_names in os.walk(source_dir):
                for file_name in tqdm(file_names, desc = f"Архивирование: {dir_path}"):
                    file_path = os.path.join(source_dir, dir_path, file_name)

                    arcname = os.path.relpath(file_path, start = "/content/")

                    zipf.write(file_path, arcname = arcname)

    # Укажите путь для архива
    send_zip = '/content/drive/MyDrive/Colab Notebooks/Datas/Zips/Karate mixed 11 classes 562 Images _ crop 224-224.zip'
    # Копирование
    shutil.copy(zip_file_path, send_zip)

    print("Архивирование и копирование завершены.")

# Список папок, которые вы хотите архивировать
source_dirs = ['/content/Karate mixed 11 classes 562 Images _ crop X-X/Karate train_224x224',
               '/content/Karate mixed 11 classes 562 Images _ crop X-X/Karate test_224x224']

# Используйте функцию для архивирования
create_zip_archive('/content/Karate mixed 11 classes 562 Images _ crop 224-224.zip', source_dirs)

Общий прогресс:   0%|          | 0/2 [00:00<?, ?it/s]

Архивирование: /content/Karate mixed 11 classes 562 Images _ crop X-X/Karate train_224x224: 0it [00:00, ?it/s]

Архивирование: /content/Karate mixed 11 classes 562 Images _ crop X-X/Karate train_224x224/Шико Дачи (Shiko Da…

Архивирование: /content/Karate mixed 11 classes 562 Images _ crop X-X/Karate train_224x224/Тсуру Аши Дачи (Tsu…

Архивирование: /content/Karate mixed 11 classes 562 Images _ crop X-X/Karate train_224x224/Неко Аши Дачи (Neko…

...

Архивирование: /content/Karate mixed 11 classes 562 Images _ crop X-X/Karate test_224x224/Дзенкуцу Дачи (Zenku…

Архивирование: /content/Karate mixed 11 classes 562 Images _ crop X-X/Karate test_224x224/Кокутсу Дачи (Kokuts…

Архивирование: /content/Karate mixed 11 classes 562 Images _ crop X-X/Karate test_224x224/Киба Дачи (Kiba Dach…

Архивирование: /content/Karate mixed 11 classes 562 Images _ crop X-X/Karate test_224x224/Мусуби дачи (Musubi …

Архивирование и копирование завершены.
